<a href="https://colab.research.google.com/github/njanrepo/MLEProject/blob/main/task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import math

In [2]:
!pip install yfinance

     |████████████████████████████████| 6.3 MB 35.0 MB/s 
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [44]:
import yfinance as yf

### Task1 :: Get companies

In [45]:
def get_nyse_ticker_list():
  others_list = 'ftp://ftp.nasdaqtrader.com/symboldirectory/otherlisted.txt'
  ticker_df = pd.read_csv(others_list, sep='|')
  company_nyse = ticker_df[ticker_df['Exchange'] == 'N']
  return company_nyse['ACT Symbol'].values.tolist()

In [46]:
def get_sp500_list():
  payload=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
  first_table = payload[0]
  sp500df = first_table
  sp500_list = sp500df.Symbol.values.tolist()
  return sp500_list

In [48]:
def get_stock_data(type='S&P500', limit=100):
  ticker_list = get_nyse_ticker_list() if type == 'NYSE' else get_sp500_list()
  if(limit > len(ticker_list)):
    limit = len(ticker_list)
  ticker_data  = yf.download(ticker_list[0:limit], period='6mo', progress=True, group_by='symbol')
  try:
    ticker_data.drop('BF.B', axis=1, inplace=True)
    ticker_data.drop('BRK.B', axis=1, inplace=True)
  except:
    print("its fine!!")
  ticks = ticker_data.columns.get_level_values(0).unique().tolist()
  return (ticker_data, ticks)




#Calculate Market Capitalization


In [7]:
#wrong, it is just volum, not total outstanding shares
ticker='JPM'
def calc_market_capitalization(symbol):
  symbol_data = yf.download(symbol, period='1d')
  print(symbol_data)
  return round((symbol_data['Volume'][0] * symbol_data['Close'][0]),2)
print(calc_market_capitalization(ticker))

[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close   Volume
Date                                                                           
2021-11-12  167.649994  167.740005  165.720001  166.860001  166.860001  8111600
1353501580.95


In [8]:
def ask_user(stocks = None):
  if(stocks != None):
    return stocks;
  how_many_stocks = input('How many stocks you want to analyse? \n Enter atleast 10 and max 500 \n')
  correct = False
  while not correct:
    try:
      how_many_stocks = int(how_many_stocks)
      if(how_many_stocks > 500 or how_many_stocks < 10):
        raise ValueError
      correct = True
    except:
      print("Please enter correct number 10 to 500")
      how_many_stocks = input('How many stocks you want to analyse? \n Enter atleast 10 and max 500 \n')
  return how_many_stocks    

In [9]:
def make_X_and_Y(close_prices, period):
  X = []
  y = []
  for i in range(len(close_prices)- period):
    X.append(close_prices[i:i+period])
    y.append(close_prices[i+period])
  return X,y

In [61]:
def cl_make_X_and_Y(close_prices, period, in_future):
  X = []
  y = []
  for i in range(len(close_prices)- period-in_future):
    X.append(close_prices[i:i+period])
    l = close_prices[i+period:i+period+in_future]
    y.append(calculate_label(l))
  return X,y

def calculate_label(params):
  changes = []
  for j in range(len(params)-1):
    changes.append(params[j+1] - params[j])
  tot = sum(changes)
  if tot > 0:
    return 1;
  elif tot < 0:
     return -1;
  else:
     return 0;


In [64]:
dt, tick = get_stock_data(limit=50)

x,y=cl_make_X_and_Y(dt['ABBV'].Close, 6, 5)


[*********************100%***********************]  50 of 50 completed
its fine!!


In [68]:
tick

['APTV',
 'ABBV',
 'AEE',
 'AAPL',
 'AMAT',
 'ARE',
 'GOOG',
 'ALGN',
 'ALB',
 'AMZN',
 'AMCR',
 'ANTM',
 'ABT',
 'AWK',
 'ANSS',
 'AXP',
 'AFL',
 'LNT',
 'ATVI',
 'AEP',
 'AIG',
 'AMGN',
 'AKAM',
 'GOOGL',
 'MMM',
 'AJG',
 'AME',
 'AMP',
 'A',
 'APA',
 'ADBE',
 'AMT',
 'APH',
 'AAL',
 'ABMD',
 'ALLE',
 'AON',
 'ANET',
 'AOS',
 'AAP',
 'ABC',
 'AES',
 'AMD',
 'ALK',
 'ADM',
 'MO',
 'ALL',
 'ADI',
 'ACN',
 'APD']

In [83]:
def get_user_selected_stocks():
  u = input("Enter ticker of stocks seperated by space..")
  return u.split(' ');
def user_selected_analysis(selected, income):
  market_cap = []
  close_price = []
  for sym in selected:
    try:
      current_data = yf.download(sym, period='1d')
      close_price.append(current_data.Close[0])
      market_cap.append(current_data.Close[0] * current_data.Volume[0])
    except:
      print('Data not found for symbol :: ', sym)
  s = sum(market_cap)
  print(s)
  stocks_to_purchase = []
  for i, cap in enumerate(market_cap):
    portion = (income* cap)/s
    print(portion)
    print(close_price[i])
    stocks_to_purchase.append(math.floor(portion/close_price[i]))
  return pd.DataFrame({'ticker': selected, 'stocks_to_purchase': stocks_to_purchase})

gft = user_selected_analysis(['ABBV', 'GOOG', 'AMZN'], 5000)
print(gft)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
12347857651.913452
129.79291767889075
116.97000122070312
1032.5512801494112
2992.909912109375
3837.6558021716983
3525.14990234375
  ticker  stocks_to_purchase
0   ABBV                   1
1   GOOG                   0
2   AMZN                   1


In [33]:
def return_model(X, y):
  clf = RandomForestRegressor(n_estimators = 200)
  clf.fit(X,y)
  return clf

In [11]:
def predict_price(symbol, period, data=None):
  print("Analysing :: ", symbol )
  if(data is None):
    data = yf.download(symbol, period='6mo', progress=False, )
  price = data.Close
  features, dep_var = make_X_and_Y(list(price), period)
  features = np.array(features)
  dep_var = np.array(dep_var)
  clf=return_model(features, dep_var) 
  
  latest_feature = np.array(list(price)[-period:]).reshape((1,-1))
  s_dict = {}
  s_dict['symbol'] = symbol
  s_dict['volatility'] = round(price.std(), 2)
  s_dict['tomorrow'] = round(clf.predict(latest_feature)[0], 2)
  s_dict['today'] = round(price[-1],2)
  return s_dict

In [14]:
predict_price('MMM', 4)

Analysing ::  MMM


{'symbol': 'MMM', 'today': 183.66, 'tomorrow': 180.42, 'volatility': 9.51}

In [23]:
def prepare_recomm(data, how_many_stocks = None, period = None):
  #list_of_symbols = get_sp500_list()
  if(data is None):
    print("No stock data found..")
    return
  list_of_symbols = data.columns.get_level_values(0).unique().tolist()
  if period == None:
    period = 6
  stk_data = []
  if how_many_stocks == None:
    how_many_stocks = ask_user()
  if how_many_stocks > len(list_of_symbols):
    print("No. of stocks set to max")  
    how_many_stocks = len(data)
  for sym in list_of_symbols[0:how_many_stocks]:
    stk_data.append(predict_price(sym, period,data[sym]))
  return pd.DataFrame(stk_data)


In [16]:
def get_recomm(s_data, ascending):
  cat = 'conservative' if ascending else 'aggressive'
  print("Top 5 " + cat +" stocks are.. ")
  tmp = s_data.sort_values(by='volatility', ascending=ascending)
  tmp = tmp[tmp['tomorrow'] > tmp['today']]
  return tmp.head()  

In [20]:
#get_stock_data takes two param, 
  #type: S&P500(default) or 'NYSE, 
  #limit: default 100
data_df, ticker_list = get_stock_data()

[*********************100%***********************]  100 of 100 completed

2 Failed downloads:
- BRK.B: No data found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted


/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:3889: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [37]:
pred_df = prepare_recomm(data_df, 500, 6)

No. of stocks set to max
Analysing ::  APTV
Analysing ::  AAPL
Analysing ::  CDNS
Analysing ::  ARE
Analysing ::  ALB
Analysing ::  ANTM
Analysing ::  AWK
Analysing ::  ABT
Analysing ::  AFL
Analysing ::  ATVI
Analysing ::  BA
Analysing ::  CZR
Analysing ::  BMY
Analysing ::  CDW
Analysing ::  AKAM
Analysing ::  BXP
Analysing ::  AVGO
Analysing ::  A
Analysing ::  AME
Analysing ::  BWA
Analysing ::  BRO
Analysing ::  AON
Analysing ::  CHRW
Analysing ::  COF
Analysing ::  ABC
Analysing ::  AES
Analysing ::  AVY
Analysing ::  ADSK
Analysing ::  CNC
Analysing ::  APD
Analysing ::  ABBV
Analysing ::  AIZ
Analysing ::  ACN
Analysing ::  AEE
Analysing ::  BAC
Analysing ::  AZO
Analysing ::  BK
Analysing ::  AMZN
Analysing ::  AXP
Analysing ::  AEP
Analysing ::  AVB
Analysing ::  CARR
Analysing ::  T
Analysing ::  TECH
Analysing ::  CTLT
Analysing ::  AMP
Analysing ::  ATO
Analysing ::  BSX
Analysing ::  BAX
Analysing ::  BR
Analysing ::  BIO
Analysing ::  ANET
Analysing ::  AAP
Analysing :: 

In [38]:
has_given_resp = False
strategy = input("Enter Strategy : \n Enter 1 for Conservative\n Enter 2 for Aggresive\n")
while not has_given_resp:
  if strategy == '1' or strategy.lower() == 'c' or strategy.lower() == 'conservative':
    has_given_resp = True
    print(get_recomm(pred_df, True))
  elif strategy == '2' or strategy.lower() == 'a' or strategy.lower() == 'aggresive':
    has_given_resp = True
    print(get_recomm(pred_df, False))
  else:
    print("Please enter correct number")
    strategy = input("Enter Strategy : \n Enter 1 for Conservative\n Enter 2 for Aggresive\n")


Enter Strategy : 
 Enter 1 for Conservative
 Enter 2 for Aggresive
2
Top 5 aggressive stocks are.. 
   symbol  volatility  tomorrow   today
50    BIO       75.79    758.55  756.28
67   ADBE       54.47    658.36  657.60
56   ALGN       49.87    689.42  687.80
43   TECH       36.34    506.51  504.50
21    AON       26.82    304.50  300.34
